# <center>3rd ML Month - 18th solution </center>

## <a id='0'><strong>운영진에 감사 말씀드립니다.</strong></a>
좋은 데이터로 재미난 대회를 만들고 이끌어주신 운영진께 감사 말씀드립니다.

덕분에 기술적으로 많은 것을 배웠고 특히 **<u>"세상은 넓고 고수는 많다"</u>**는 것을 다시 한번 느끼게 되었습니다.

특히 얼마전 캐글 입문 강의로 수고해주신 태진님께 다시 한번 감사 말씀드립니다.      
       
앞으로도 이런 좋은 대회가 이어지면 좋겠습니다.
       
(비록 자랑할만한 등수는 아니지만 고수님들의 커널을 보고 배우며 삽질했던 내용을     
공유하면 좋을 것 같아서 남깁니다.)


### **Model Summary**
- Resnext101 8 Folds로 최종 제출했고 대회 종료 이틀전에 큰 오류가 있음을 깨닫고 급하게 수정하고<br>
  학습시키다보니 다른 모델을 제대로 앙상블 시킬 시간은 없었습니다.
- Public 27위→Private 18위로 올라선 것은 TTA+8folds로 Generalization이 됐기 때문인듯합니다.  
       ① 전처리  : Cropping , Histogram Equalization
       ② Augmentation : cutout , rotation 30, horizonal=True, zoom:0.3, rescale :1/255
       ③ Loss Function : categorical_crossentropy
       ④ optimizer : Adam optimizer
       ⑤ Basic Model : Resnext101(imagenet pretrained), 8 folds (평균 5.5hr/1fold 소요)
       ⑥ Inference 시 TTA(5) 적용
  

## <strong>삽질의 기록</strong>
**1.  Resnet50 + cropping + sigle fold** (public score 0.8345) <br>
**2.  Resnet50 + cropping + 5folds** (public score 0.91439) <br>
       - 학습이 제대로 되지 않고 있는 것으로 판단하고 뜯어보기 시작
           . learning rate : 0.001 → 0.0001
           . Dropout(0.25) 추가
**3.  Resnet50 + cropping + 5folds** (public score 0.92061) <br>
        - 깊은 모델(resnet50 → resnext101)로 변경하기로 결정
        - resnet101,resnext101 모델은 keras.applications가 아닌 keras_applications 모듈에서
          가져와야함. keras가 처음이라 이걸 몰라서 삽질을 오래함.
        - keras_applications 에서 resnext101를 부르면 아래 에러가 생기는데,
          모델을 부를 때 arguments를 넣어 줘야함(backend, layers, models, utils)         
         "AttributeError: 'NoneType' object has no attribute 'image_data_format'"
         →해결책 : http://donghao.org/2019/02/22/using-resnext-in-keras-2-2-4/
        - 다음에 마주하게 되는 에러가 scaling과 관련된 에러인데 이는 preprocess_input을 활용하기
          때문에 발생하는 것으로 Data Augmentation을 할 때 scale : 1/255로 변경하면 해결됨.
**4.  Resnext101 + cropping + 5folds** (public score 0.93591) <br>    
        - early stopping을 f1 score에 걸어두어서인지 val_loss가 충분히 떨어지지 않았는데
          학습이 종료되는 현상이 발생함(underfitting) 
          . patience = 5 → 10 변경
          . ReduceLROnPlateau(factor 0.5) 적용
              → 뒤에 설명하겠지만 막판에 여기에 오류가 있다는 것을 알게되어 학습을 다시함.
         - 학습 시간이 3배로 늘어나면서 Training / Inference kernel을 분리하기 시작.
         
**5.  Resnext101 + cropping + 5folds+ReduceLR** (public score 0.94787)          
         - train Loss가 0에 가깝고 acc도 100에 가까운 반면, Val acc는 98% 수준으로 
           Overfitting 되고 있다고 판단 (실제 Private score 0.943으로 Drop)
           . Mixup 적용(0.2)했으나 오히려 Public score 0.01이상 drop
           . overfitting 된 model 5개(+ Xception, resnet101, resnet50, Efficient_v3)의 결과물로
             Voting을 수행하여 Public score 0.95074를 만들었으나 private score는 저조(0.946)
             → Voting 보다 softmax 결과물들의 평균내어 argmax를 취하는게 더 성능이 좋음
         - ** 5일간 여러 삽질을 하는 도중 두가지 큰 오류가 있음을 확인 **
            . ReduceLROnPlateau 오류 : f1 score를 바라보게 해두고 mode를 default(min)으로 설정되어<br>
            (patience 10) f1 score와 관계 없이 계속 절반씩 줄고 있었음. 
            . Earlystopping 오류 : monitoring을 val_f1_score가 아닌 f1_score로 설정해두어<br>
              validation set이 아닌 train set의 f1 score가 더이상 개선이 없을때 끝나도록 되어 있었음<br>
              사실상 Earlystopping 기능을 수행하지 못하고 Overfitting되고 있음
**6.  (오류수정) Resnext101 + cropping + 8folds+ReduceLR** (public score 0.95217)
        - 위 오류를 모두 수정하여 학습하기 시작, 대회 종료까지 하루밖에 남지 않은 상황이라<br>
            한번에 학습 기회가 있을 듯하여 folds 수도 늘려서 조금이라도 overfitting을 줄여보고자함.
        - 학습이 종료된 이후에 voting을 수행하여 score를 조금더 올렸으나 private score로 보면<br>
          voting 하기 전이 더 잘 나옴


In [ ]:
import gc
import os 
import warnings
warnings.filterwarnings(action='ignore')
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm # 진행 상태 표시as
from keras import backend as K
K.image_data_format() # 채널 first 인지 last인지 여부 판단
# Image visualization

import PIL
from PIL import ImageDraw

In [ ]:
path = '../input/2019-3rd-ml-month-with-kakr'
os.listdir(path)
# 이미지 폴더 경로 
train_img_path = os.path.join(path,'train')
test_img_path = os.path.join(path,'test')
# csv 파일 경로
df_train = pd.read_csv(os.path.join(path,'train.csv'))
df_test = pd.read_csv(os.path.join(path,'test.csv'))
df_class = pd.read_csv(os.path.join(path,'class.csv'))

In [ ]:
#crop

def crop_boxing_img(img_name, margin=-4, size=(224,224)):
    if img_name.split('_')[0] == 'train':
        PATH = train_img_path
        data = df_train
    else:
        PATH = test_img_path
        data = df_test

    img = PIL.Image.open(os.path.join(PATH, img_name))
    pos = data.loc[data["img_file"] == img_name, ['bbox_x1', 'bbox_y1', 'bbox_x2', 'bbox_y2']].values.reshape(-1)

    width, height = img.size
    x1 = max(0, pos[0] - margin)
    y1 = max(0, pos[1] - margin)
    x2 = min(pos[2] + margin, width)
    y2 = min(pos[3] + margin, height)

    return img.crop((x1, y1, x2, y2)).resize(size)

In [ ]:
TRAIN_CROPPED_PATH = '../cropped_train'
TEST_CROPPED_PATH = '../cropped_test'

In [ ]:
if (os.path.isdir(TRAIN_CROPPED_PATH) == False):
    os.mkdir(TRAIN_CROPPED_PATH)

if (os.path.isdir(TEST_CROPPED_PATH) == False):
    os.mkdir(TEST_CROPPED_PATH)

for i, row in df_train.iterrows():
    cropped = crop_boxing_img(row['img_file'])
    cropped.save(os.path.join(TRAIN_CROPPED_PATH, row['img_file']))

for i, row in df_test.iterrows():
    cropped = crop_boxing_img(row['img_file'])
    cropped.save(os.path.join(TEST_CROPPED_PATH, row['img_file']))

In [ ]:
# Set Path of Preprocessed Train Images
TRAIN_IMG_PREP_PATH = os.path.join('..', 'train_prep')
if not os.path.exists(TRAIN_IMG_PREP_PATH):
    os.makedirs(TRAIN_IMG_PREP_PATH, exist_ok=True)

# Set Path of Preprocessed Test Images
TEST_IMG_PREP_PATH = os.path.join('..', 'test_prep')
if not os.path.exists(TEST_IMG_PREP_PATH):
    os.makedirs(TEST_IMG_PREP_PATH, exist_ok=True)

In [ ]:
def img_he_pad(img_file_name, add_padding=True):
    if img_file_name.split('_')[0] == 'train':
        IMG_CROP_PATH = TRAIN_CROPPED_PATH
        data = df_train
    elif img_file_name.split('_')[0] == 'test':
        IMG_CROP_PATH = TEST_CROPPED_PATH
        data = df_test
        
    # --- Histogram Equalization --- #
    img = cv2.imread(os.path.join(IMG_CROP_PATH, img_file_name))
    img_y_cr_cb = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
    y, cr, cb = cv2.split(img_y_cr_cb)

    # Equalize y (CLAHE (Contrast Limited Adaptive Histogram Equalization))
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(6,6))
    y_eq = clahe.apply(y)
    img_y_cr_cb_eq = cv2.merge((y_eq, cr, cb))
    img_bgr_eq = cv2.cvtColor(img_y_cr_cb_eq, cv2.COLOR_YCR_CB2BGR)
    img_prep = img_bgr_eq

    # --- Convert BGR To RGB (Just On cv2) ---
    # b, g, r = cv2.split(img_bgr_eq)
    # img_prep = cv2.merge((r,g,b))
    
    # -------- Add Padding --------- #
    if add_padding:
        img_prep_h, img_prep_w = img_prep.shape[0], img_prep.shape[1]  # (height, width)
        ratio = float(IMG_SIZE) / max(img_prep_h, img_prep_w)
        shape_no_padding = (int(img_prep_h * ratio), int(img_prep_w * ratio))

        img_prep_no_padding = cv2.resize(img_prep, shape_no_padding[::-1])
        
        size_h = IMG_SIZE - shape_no_padding[0]
        size_w = IMG_SIZE - shape_no_padding[1]
        
        top, bottom = size_h // 2, size_h - (size_h // 2)
        left, right = size_w // 2, size_w - (size_w // 2)

        PADDING_COLOR = (0, 0, 0)  # black
        img_prep = cv2.copyMakeBorder(
            img_prep_no_padding,
            top,
            bottom,
            left,
            right,
            cv2.BORDER_CONSTANT,
            value=PADDING_COLOR
        )
    
    return img_prep

In [ ]:
import cv2
IMG_SIZE = 224
# Save Preprocessed Train Images (Path: ../train_prep)
if not os.listdir(TRAIN_IMG_PREP_PATH):  # If PATH_IMG_TRAIN_PREP is empty
    for idx, row in df_train.iterrows():
        img_file_name = row['img_file']
        img_prep = img_he_pad(img_file_name, add_padding=True)
        cv2.imwrite(os.path.join(TRAIN_IMG_PREP_PATH, img_file_name), img_prep)

# Save Preprocessed Test Images (Path: ../test_prep)
if not os.listdir(TEST_IMG_PREP_PATH):
    for idx, row in df_test.iterrows():
        img_file_name = row['img_file']
        img_prep = img_he_pad(img_file_name, add_padding=True)
        cv2.imwrite(os.path.join(TEST_IMG_PREP_PATH, img_file_name), img_prep)

In [ ]:
# Define Function For Test
def test_he_padding(img_file_name):
    # Show Cropped Image
    img_crop = PIL.Image.open(os.path.join(TRAIN_CROPPED_PATH, img_file_name))
    plt.figure(figsize=(12, 9))
    plt.subplot(1, 2, 1)
    plt.title(f'Cropped Image - {img_file_name}')
    plt.imshow(img_crop)
    plt.axis('off')

    # Show Preprocessed Image
    img_he_pad = PIL.Image.open(os.path.join(TRAIN_IMG_PREP_PATH, img_file_name))
    plt.subplot(1, 2, 2)
    plt.title(f'Historgram Equalized Cropped Image(Add Padding) - {img_file_name}')
    plt.imshow(img_he_pad)
    plt.axis('off')
    
    # Show Result
    plt.show()
    
# Test Histogram Equalization & Add Padding
test_he_padding(img_file_name=df_train['img_file'].iloc[114])

In [ ]:
from sklearn.metrics import f1_score

def f1_metric(y_true, y_pred):

    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = (true_positives + K.epsilon()) / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = (true_positives + K.epsilon()) / (predicted_positives + K.epsilon())
        return precision

    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
from keras_applications.resnext import ResNeXt101#, preprocess_input
# from keras.applications.resnet_v2. import ResNet50, preprocess_input
# from keras_applications.resnet import ResNet101, preprocess_input
from keras.preprocessing.image import ImageDataGenerator

# Parameter
img_size = (224, 224)
epochs = 70
batch_size =16

# Define Generator config
train_datagen = ImageDataGenerator(
    rotation_range=30,
    horizontal_flip = True, 
    vertical_flip = False,
    #zoom_range=0.30,
    #width_shift_range=0.2,
    #height_shift_range=0.2,
    #shear_range=0.5,
    brightness_range=[0.5, 1.5],
    fill_mode='nearest',
    rescale=1./255)
    #preprocessing_function=preprocess_input)

val_datagen = ImageDataGenerator(rescale=1./255)#preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(rescale=1./255)#preprocessing_function=preprocess_input)

In [ ]:
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, GlobalAveragePooling2D
from keras import layers, models, optimizers, utils, backend,regularizers
# import keras
def get_model(model_name='ResNeXt101'):
    resNet_model = ResNeXt101(include_top= False, input_shape = (224,224,3)
                            , backend =backend, layers=layers, models = models,
                             utils = utils
                            )
    # resNet_model.summary()
    
    model = Sequential()
    model.add(resNet_model)
    model.add(GlobalAveragePooling2D())
    model.add(layers.Dropout(0.25))  # 과적합 줄여보기
    model.add(Dense(196, activation='softmax', kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(l=0.01)))
#     model.add(Dense(196, activation='softmax', kernel_initializer='he_normal'))
#     model.add(Dense(196, kernel_initializer='he_normal', kernel_regularizer=regularizers.l2(l=0.01)))
#     model.add(LeakyReLU(alpha=0.01))    
    model.summary()
    adam = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(optimizer = adam,loss = 'categorical_crossentropy', metrics=['acc',f1_metric])
    # compile 할때 넣어줘야지 아래에서 early stopping 할때 사용 가능하다
    return model



In [ ]:
def get_steps(num_samples,batch_size):
    if (num_samples % batch_size)>0:
        return (num_samples // batch_size) + 1
    else :
        return num_samples // batch_size

In [ ]:
from sklearn.model_selection import StratifiedKFold
k_folds = 5
kfold =StratifiedKFold(n_splits = k_folds, random_state = 1990)
df_train['class'] = df_train['class'].astype('str')
df_train= df_train[['img_file','class']]
df_test = df_test[['img_file']]

In [ ]:
%%time
file=[]
from keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau
for idx,(train_index, valid_index) in enumerate(kfold.split(
                                df_train['img_file'],df_train['class'])):
    if idx != 0 : continue # 여러번 나눠서 돌리기 위함
   # if idx == 1 : continue
    #if idx == 2 : continue
    #if idx == 3 : continue    
    traindf = df_train.iloc[train_index,:].reset_index()
#     validdf = df_train.iloc[valid_index,:].reset_index()
#     traindf.to_csv('%s_traindf'%idx,index=False)
#     validdf.to_csv('%s_validdf'%idx,index=False)
    
    nb_train_samples = len(traindf)
#     nb_validation_samples = len(validdf)
#     nb_test_samples = len(df_test)
    # Make Generator
    train_generator = train_datagen.flow_from_dataframe(
        dataframe=traindf, 
        directory=TRAIN_CROPPED_PATH,#'../input/train/',
        x_col = 'img_file',
        y_col = 'class',
        target_size = img_size,
        color_mode='rgb',
        class_mode='categorical',
        batch_size=batch_size,
        seed=42
    )





In [ ]:

test_generator = test_datagen.flow_from_dataframe(
            dataframe=df_test,
            directory=TEST_IMG_PREP_PATH,#TEST_CROPPED_PATH,#'../input/test',
            x_col='img_file',
            y_col=None,
            target_size= img_size,
            color_mode='rgb',
            class_mode=None,
            batch_size=batch_size,
            shuffle=False
            )

In [ ]:
path = '../input/models10/'
lst = os.listdir(path)
print(lst)

In [ ]:

%%time
tta_steps = 5 #models5 기준 1회 0.950 ,5회 0.949 , 10회 0.9507
prediction = []
for i, name in enumerate(lst):
    preds =[]
    print(name)
    model = get_model()
    model.load_weights(os.path.join(path,name))
    for j in tqdm(range(tta_steps)):                
        test_generator.reset()
        nb_test_samples = len(df_test)
        pred = model.predict_generator(
            generator = test_generator,
            steps = get_steps(nb_test_samples, batch_size),
            verbose=1
            )
        preds.append(pred)
        
        gc.collect()
#         print(np.mean(preds,axis=0))    
    pd.DataFrame(np.mean(preds,axis=0)).to_csv('%s.csv'%i, index= False)
#     prediction.append(np.mean(preds,axis=0)) 
    del preds
    gc.collect()
for i, name in enumerate(lst):
    prediction.append(np.array(pd.read_csv('%s.csv'%i)))
# print(prediction)
y_pred = np.mean(prediction,axis=0)
print(y_pred)

In [ ]:
path = '../input/2019-3rd-ml-month-with-kakr'
preds_class_indices = np.argmax(y_pred,axis =1)
labels = (train_generator.class_indices)
labels = dict((v,k) for k,v in labels.items())
final_pred = [labels[k] for k in preds_class_indices]

submission = pd.read_csv(os.path.join(path, 'sample_submission.csv'))
submission["class"] = final_pred
submission.to_csv("submission_rev03.csv", index=False)
submission.head()
              